importing necessary *libraries*

In [11]:
import numpy as np
import pandas as pd
import re #regular expression libraray
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [14]:
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv("/content/drive/MyDrive/twitter sentiment analysis using logistic regression/training.1600000.processed.noemoticon.csv", encoding='-ISO-8859-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
data.head()
print(data["text"].dtype)

object


In [18]:
columns=["target","ids","date","flag","user","text"]
data=pd.read_csv("/content/drive/MyDrive/twitter sentiment analysis using logistic regression/training.1600000.processed.noemoticon.csv", encoding='-ISO-8859-1',names=columns)
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [24]:
data["target"].value_counts()

,count
target,
0,800000
4,800000


In [25]:
data.replace([4],[1],inplace=True)

text preprocessing

In [26]:
ps=PorterStemmer()
def stemming(text):
  stemming_text=text.lower()
  stemming_text="".join([c for c in text if c.isalnum() or c.isspace()]) #removing the special characters
  stemming_text=stemming_text.split()
  stemming_text=[ps.stem(word) for word in stemming_text if word not in stopwords.words('english')]
  stemming_text=" ".join(stemming_text)
  return stemming_text
data["stemmed_data"]=data["text"].apply(stemming)

In [ ]:
X=data["stemmed_data"].values
y=data["target"].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

creating ML model logistic regression


In [28]:
print(X_train)
print(X_test)

  (0, 635023)	0.27185619477401424
  (0, 526338)	0.3572368601721922
  (0, 306051)	0.5289558736505798
  (0, 155665)	0.37500971658120597
  (0, 364397)	0.4201488011008131
  (0, 644304)	0.4489147093057786
  (1, 222401)	0.9735756315752642
  (1, 296625)	0.22836481691106764
  (2, 155665)	0.46221360915332715
  (2, 296625)	0.11805968011878909
  (2, 174051)	0.19049806487955614
  (2, 595380)	0.18832213903949507
  (2, 181431)	0.291592042723569
  (2, 593982)	0.32252824840643696
  (2, 630938)	0.3320802314498419
  (2, 114952)	0.31554393572550504
  (2, 644838)	0.33684882171015035
  (2, 404837)	0.24379038813702913
  (2, 598071)	0.1531204789500814
  (2, 208534)	0.1889218766287001
  (2, 185024)	0.20356605788086823
  (2, 432917)	0.1682044181233032
  (3, 593982)	0.2897156760024441
  (3, 219764)	0.4472269018910956
  (3, 210185)	0.27802924118981875
  :	:
  (1279996, 573824)	0.21775605929636588
  (1279996, 550927)	0.3569565396691649
  (1279996, 368229)	0.5364861194239735
  (1279996, 368231)	0.5520640451393458


In [31]:
# creating the model
model=LogisticRegression(max_iter=1000)

In [33]:
model.fit(X_train,y_train)
X_train_prediction=model.predict(X_train)
X_train_accuracy=accuracy_score(X_train_prediction,y_train)
print(X_train_accuracy)
X_test_prediction=model.predict(X_test)
X_test_accuracy=accuracy_score(X_test_prediction,y_test)
print(X_test_accuracy)

0.809809375
0.78486875


In [36]:
#to save the model
import pickle
model_name="twitter_analysis-model.sav"
pickle.dump(model,open("twitter_analysis_model.sav","wb"))


In [42]:
# to load the model
loaded_model=pickle.load(open("twitter_analysis_model.sav","rb"))
user_data=input("enter the tweet:")
user_data=stemming(user_data)
user_data=[user_data]
user_data=vectorizer.transform(user_data)
prediction=loaded_model.predict(user_data)
if prediction[0]==0:
  print("negative TWEET")
else:
  print("positive TWEET")

enter the tweet:it is too vulgur
positive TWEET
